In [1]:
import numpy as np
import pandas as pd
import cufflinks as cf
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, confusion_matrix, roc_auc_score,plot_roc_curve
import matplotlib.pyplot as plt
cf.go_offline()
pd.set_option('display.max_columns',300)
pd.set_option('display.max_rows',300)

In [ ]:
df=pd.read_pickle("Tabla_Proyecto_Limpia_forma1.pkl")

In [ ]:
df=df[df["Source"]=="MapQuest"].copy()

In [ ]:
quitar=['TMC','Start_Time', 'End_Time', 'Description', 'Street','Side','City','County','Zipcode',
          'Airport_Code','Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)',
       'Wind_Direction', 'Wind_Speed(mph)', 'Weather_Condition','City_Freq','Weather', 'Temperature(F)_Escalado', 'Humidity(%)_Escalado',
       'Pressure(in)_Escalado', 'Visibility(mi)_Escalado',
       'Wind_Speed(mph)_Escalado','Day']

In [ ]:
columnas=[x for x in df.columns if x not in quitar]

In [ ]:
df=df[columnas]

In [ ]:
df["Amenity"]=df["Amenity"]*1
df["Crossing"]=df["Crossing"]*1
df["Junction"]=df["Junction"]*1
df["Station"]=df["Station"]*1
df["Stop"]=df["Stop"]*1
df["Traffic_Signal"]=df["Traffic_Signal"]*1
df["Hour"]=df["Hour"].astype(str)
df["Year"]=df["Year"].astype(str)
df["Side_moda"]=(df["Side_moda"]=="R")*1
df["Weekend"]=df["Weekday"].map(lambda x: 1 if x=="Sat" or x=="Sun" else 0)

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
cat=["State","Timezone","Hour","Weekend",
     "Month","Side_moda","Wind_Direction_moda","Weather_moda"]
cont=[x for x in df.columns if x not in cat+["Severity","Source","Distance(mi)","Weekday",
                                            "Year","TimeMinutes"]]

In [ ]:
dic={1:0,2:0,3:1,4:1}
df["target"]=df["Severity"].replace(dic)

In [ ]:
X = df[cont].join(pd.get_dummies(df[cat]))
y = df["target"]

In [ ]:
columnas_mejor_modelo=['Start_Lat', 'Amenity', 'Crossing','Junction', 'Station',
 'Stop', 'Traffic_Signal', 'State_Freq', 'Airport_Freq', 'Weekend', 'Side_moda',
 'State_AZ', 'State_CA', 'State_CT', 'State_GA', 'State_IL', 'State_LA', 'State_MN',
 'State_MO', 'State_NC', 'State_OK', 'State_RI', 'State_SC', 'State_TX','State_VA',
 'Timezone_US/Central', 'Timezone_US/Pacific', 'Hour_7', 'Hour_8', 'Weather_moda_Fair']

In [ ]:
X=X[columnas_mejor_modelo]

In [ ]:
modelo=pd.read_pickle("modelos/regresion_logistica.pickle")

In [ ]:
modelo

In [ ]:
list(zip(columnas_mejor_modelo,modelo.coef_[0]))

In [ ]:
modelo.score(X, y)

In [ ]:
df["proba"] = modelo.predict_proba(X)[:, 1]

In [ ]:
df["y_hat"] = modelo.predict(X)

In [ ]:
aux = df[["target", "y_hat", "proba"]]

In [ ]:
target="target"

In [ ]:
df_res = pd.DataFrame()
for i in range(100):
    try:
        tn, fp, fn, tp = confusion_matrix(aux.loc[aux["proba"]>=i/100, target], aux.loc[aux["proba"]>=i/100, "y_hat"]).ravel()
        df_res.loc[i/100, "specificity"] = 1- tn / (fp+tn)
        df_res.loc[i/100, "sensitivity"] = tp / (tp+fn)
    except:
        df_res.loc[i/100, "specificity"] = 0
        df_res.loc[i/100, "sensitivity"] = 0

In [ ]:
df_res

In [ ]:
df_res.sort_values(by = "sensitivity").set_index("specificity").iplot(kind="scatter", mode="markers+lines")

In [ ]:
df[["target","Distance(mi)"]].groupby(by="target").agg("mean")

# Continuas

In [ ]:
df=pd.read_csv("modelos/Columnas_modelo_continuo_timeminutes.csv")

In [ ]:
modelo=pd.read_pickle("modelos/regresion_Ridge.pickle")

In [ ]:
df["coeficientes"]=modelo.coef_

In [ ]:
df.sort_values(by="coeficientes")

In [ ]:
df=pd.read_pickle("Tabla_Proyecto_Limpia_forma1.pkl")
df=df[df["Source"]=="MapQuest"].copy()

In [ ]:
aux=df[["Weather_moda","TimeMinutes"]].groupby(by=["Weather_moda"]).agg("mean")

In [ ]:
aux=aux.sort_values(by="TimeMinutes", ascending=False)

In [ ]:
aux.iplot(kind='bar',title='Tiempo promedio de los accidentes',color=['blue'],
                            theme='white',yTitle='Minutos',xTitle='Clima')

In [2]:
modelo=pd.read_pickle("modelos/credit_scoring.pkl")

In [3]:
modelo

LogisticRegression(C=0.05, solver='saga', tol=0.01)

In [4]:
variables_usadas_en_los_modelos=['Side_moda', 'Traffic_Signal', 'State', 'Crossing', 'Weekday',
       'Junction', 'Stop', 'Hour', 'Timezone', 'Amenity', 'Station',
       'Start_Lng', 'Weather_moda']

In [5]:
list(zip(variables_usadas_en_los_modelos,modelo.coef_[0]))

[('Side_moda', -0.9036372867263275),
 ('Traffic_Signal', -0.7830803532227957),
 ('State', -0.7869853425180692),
 ('Crossing', -0.4306838278877023),
 ('Weekday', -0.7261759599322303),
 ('Junction', -0.43636341194058703),
 ('Stop', -0.8813722597006128),
 ('Hour', -0.8338485202977018),
 ('Timezone', 0.5303164174695902),
 ('Amenity', -0.5765353825918578),
 ('Station', -0.4696991690009925),
 ('Start_Lng', -0.014704279495636692),
 ('Weather_moda', -0.8582557841735139)]